In [1]:
import copy
import math

# 外皮情報LV3からLV4へのコンバート

## Outline

- 外皮性能計算プログラムに、外皮の用途別情報を追加したものより、内壁等（建物内部の壁、天井、床）の面積の推定および熱貫流率の決定を行う。
- 外皮の用途別情報が入力された場合において床下空間に属する土間床等面積が0㎡より大きい場合、室と床下空間の間の床の面積の合計は、床下空間に属する土間床等面積の合計（ただし、室用途別の床面積と方位が「下面」の部位面積の差分の合計を上限とする）に等しいとみなす。室と床下空間の間の床は、室用途別の床面積と方位が「下面」の部位面積の差分に応じて、各用途に割り当てる。
- 室と床下空間の間の床は、仕様を想定して与える（合板12mm、参考：SimHeatでの熱負荷計算用の無断熱の層構成）。
- 内壁等の面積は、室の周長を求める際の係数（アスペクト比）と階高を仮定して、外皮の用途別情報および用途別床面積より推定する。主たる居室―非居室間、その他居室―非居室間は、それぞれ、各用途の床面積が0㎡より大きい場合は、居室と非居室は幅1m、高さ2m以上の通路等で繋がっていると想定する。
- 内壁等の熱貫流率は一般的な仕様を想定して与える（石膏ボード12.5mm－空気層－石膏ボード12.5mm、参考：SimHeatでの熱負荷計算用の無断熱の層構成）。室内戸は考慮しない。
- その他は、内壁の用途別情報が得られている場合と同様に求める。

## 1. Common Element

#### Outline

'Common'要素はそのまま維持する。

参照：InputDataDefinition.ipynb

#### Function

In [2]:
def convert_common(d):
    return d['Common']

## 2. Walls Element

#### Outline

'Walls'要素はそのまま維持する。

#### Function

In [3]:
def convert_wall(d):
    return d['Walls']

## 3. Windows Element

#### Outline

'Windows'要素はそのまま維持する。

#### Function

In [4]:
def convert_window(d):
    return d['Windows']

## 4. Doors Element

#### Outline

'Doors'要素はそのまま維持する。

#### Function

In [5]:
def convert_door(d):
    return d['Doors']

## 5. Heatbridges Element

#### Outline

'Heatbridges'要素はそのまま維持する。

#### Function

In [6]:
def convert_heatbridge(d):
    return d['Heatbridges']

## 6. EarthfloorPerimeters Element

#### Outline

'EarthfloorPerimeters'要素はそのまま維持する。

#### Function

In [7]:
def convert_earthfloorperimeter(d):
    return d['EarthfloorPerimeters']

## 7. Earthfloors Element

#### Outline

'Earthfloors'要素はそのまま維持する。

#### Function

In [8]:
def convert_earthfloor(d):
    return d['Earthfloors']

## 8. InnerWalls Element

## 8.1. 用途別床面積を取得する関数

In [9]:
def get_area_floor(d):
    return d['Common']['MainOccupantRoomFloorArea'], \
           d['Common']['OtherOccupantRoomFloorArea'], \
           d['Common']['TotalFloorArea'] - d['Common']['MainOccupantRoomFloorArea'] - d['Common']['OtherOccupantRoomFloorArea']

## 8.2. InnerWalls Element - Inner Floor 

### 1) 層構成

#### Outline

以下の層構成とする。

| 材料 | 厚さ, m | 熱伝導率, W m<sup>-1</sup> K<sup>-1</sup> | 熱抵抗, m<sup>2</sup> k W<sup>-1</sup> | 容積比熱, J L<sup>-1</sup> K<sup>-1</sup> |
|---|---|---|---|
| 合板 | 0.012 | 0.16 | 0.075 | 720.0 |

#### Function

In [10]:
def get_inner_floor_layers():
    return [{'Name': 'PED', 'Cond': 0.16, 'Thick': 0.012, 'SpecH': 720.0}]

### 2) 床下空間上の床面積の推定

#### Outline

$$
\displaystyle
A_{f,MR}= A_f \times \frac{ A_{dif,MR} }{ A_{dif,MR} + A_{dif,OR} + A_{dif,NR} }
$$

$$
\displaystyle
A_{f,OR}= A_f \times \frac{ A_{dif,OR} }{ A_{dif,MR} + A_{dif,OR} + A_{dif,NR} }
$$

$$
\displaystyle
A_{f,NR}= A_f \times \frac{ A_{dif,NR} }{ A_{dif,MR} + A_{dif,OR} + A_{dif,NR} }
$$

$A_{f,MR}$: 主たる居室と床下空間の間の床の面積, m<sup>2</sup>  
$A_{f,OR}$: その他の居室と床下空間の間の床の面積, m<sup>2</sup>  
$A_{f,NR}$: 非居室と床下空間の間の床の面積, m<sup>2</sup>  
$A_f$: 室と床下空間の間の床面積の合計, m<sup>2</sup>  
$A_{dif,MR}$: 主たる居室において、方位が「下面」の部位の面積と床面積の差, m<sup>2</sup>  
$A_{dif,OR}$: その他の居室において、方位が「下面」の部位の面積と床面積の差, m<sup>2</sup>  
$A_{dif,NR}$: 非居室において、方位が「下面」の部位の面積と床面積の差, m<sup>2</sup>  

$$
\displaystyle
A_f= min(A_{dif,MR} + A_{dif,OR} + A_{dif,NR}, \sum_{i}^{}A_{b,i})
$$

$A_{b,i}$: 床下空間に属する土間床等の部位$i$の面積, m<sup>2</sup>

$$
\displaystyle
A_{dif, MR} = max(0, A_{MR} - \sum_i A_{of,MR,i})
$$

$$
\displaystyle
A_{dif, OR} = max(0, A_{OR} - \sum_i A_{of,OR,i})
$$

$$
\displaystyle
A_{dif, NR} = max(0, A_{NR} - \sum_i A_{of,NR,i})
$$

$A_{MR}$: 主たる居室の床面積, m<sup>2</sup>  
$A_{OR}$: その他の居室の床面積, m<sup>2</sup>  
$A_{NR}$: 非居室の床面積, m<sup>2</sup>   
$A_{of,MR,i}$: 方位が「下面」である主たる居室の外皮の部位(一般部位又は開口部)$i$の面積, m<sup>2</sup>  
$A_{of,OR,i}$: 方位が「下面」であるその他の居室の外皮の部位(一般部位又は開口部)$i$の面積, m<sup>2</sup>  
$A_{of,NR,i}$: 方位が「下面」である非居室の外皮の部位(一般部位又は開口部)$i$の面積, m<sup>2</sup>  

#### Function

In [11]:
def get_area_floor_over_underfloor(d):
    
    # 方位が「下面」である居室の外皮の部位の面積の合計
    def get_A_of(d, s):
        # s : space name
        return sum(y['area'] for y in d['Walls'] + d['Windows'] + d['Doors'] if y['space'] == s and y['direction'] == 'Bottom')
    
    # 床下空間に属する土間床等の面積の取得
    def get_A_b(d):
        return sum(earthfloor['area'] for earthfloor in d['Earthfloors'] if earthfloor['space'] == 'underfloor')
    
    # 用途別床面積の取得
    A_MR, A_OR, A_NR = get_area_floor(d)
    
    # 下面（direction:Bottom）の壁・窓・ドア面積の取得
    A_of_MR, A_of_OR, A_of_NR = get_A_of(d, 'main'), get_A_of(d, 'other'), get_A_of(d, 'nonliving')
    
    # 方位が「下面」の部位の面積と床面積の差
    A_dif_MR, A_dif_OR, A_dif_NR = max(0.0, A_MR - A_of_MR), max(0.0, A_OR - A_of_OR), max(0.0, A_NR - A_of_NR)

    A_dif = A_dif_MR + A_dif_OR + A_dif_NR

    # 床下空間に属する土間床等の面積の取得
    A_b = get_A_b(d)

    #  室と床下空間の間の床面積の推定
    A_f = min(A_dif, A_b)

    # 居室と床下空間との間の床の面積
    A_f_MR, A_f_OR, A_f_NR = A_f * A_dif_MR / A_dif, A_f * A_dif_OR / A_dif, A_f * A_dif_NR / A_dif
   
    return {
        'main' : A_f_MR,
        'other' : A_f_OR,
        'nonliving' : A_f_NR
    }

#### Example

In [12]:
d = {
    'Common': {
        'Region': 6,
        'MainOccupantRoomFloorArea': 30.0,
        'OtherOccupantRoomFloorArea': 60.0,
        'TotalFloorArea': 120.0
    },
    'Walls': [
        { 'area': 60, 'direction': 'Top', 'name': 'Ceiling', 'space': 'main', 'structure': 'wood', 'type': 'ceiling' },
        { 'area': 15, 'direction': 'Bottom', 'name': 'Floor', 'space': 'other', 'structure': 'wood', 'type': 'floor' },
    ],
    'Windows': [
        { 'area': 30, 'direction': 'SW', 'name': 'WindowSW', 'space': 'main', 'type': 'single' },
        { 'area': 75, 'direction': 'Bottom', 'name': 'WindowBottom', 'space': 'main', 'type': 'single' },
    ],
    'Doors': [
        { 'area': 15, 'direction': 'NW', 'name': 'DoorNW', 'space': 'nonliving' },
        { 'area': 15, 'direction': 'NE', 'name': 'DoorNE', 'space': 'other' }
    ],
    'Earthfloors': [
        { 'area': 5, 'name': 'other', 'space': 'underfloor' },
        { 'area': 5, 'name': 'entrance', 'space': 'underfloor' }
    ]
}

get_area_floor_over_underfloor(d)

{'main': 0.0, 'other': 6.0, 'nonliving': 4.0}

### 3) 室と床下空間の間の床の構築

#### Function

In [13]:
def get_innerwall_floor(d):
    
    # 室と床下空間の間の床面積の推定
    areas_innerfloor = get_area_floor_over_underfloor(d)
    
    # 壁体名称、部位面積を更新し、「壁体が属する空間種類」「隣接空間種類」を追加する
    ifs = []
    
    ifs.extend([ {
        'Name' : 'Innerfloor_' + space,
        'Type' : 'InnerFloor',
        'Direction' : 'Horizontal',
        'Area' : areas_innerfloor[space],
        'Layers' : get_inner_floor_layers(),
        'Space' : space,
        'NextSpace' : 'Underfloor'
    } for space in ['main', 'other', 'nonliving'] ])

    ifs.extend([ {
        'Name' : 'Innerfloor_' + space,
        'Type' : 'InnerFloor',
        'Direction' : 'Horizontal',
        'Area' : areas_innerfloor[space],
        'Layers' : get_inner_floor_layers(),
        'Space' : 'Underfloor',
        'NextSpace' : space
    } for space in ['main', 'other', 'nonliving'] ])
        
    return ifs

## 8.2. InnerWalls Element - Inner Wall 

### 1) 層構成

以下の層構成とする。

| 材料 | 厚さ, m | 熱伝導率, W m<sup>-1</sup> K<sup>-1</sup> | 熱抵抗, m<sup>2</sup>2 k W<sup>-1</sup> | 容積比熱, J L<sup>-1</sup> K<sup>-1</sup> |
|---|---|---|---|
| せっこうボード | 0.0125 | 0.22 | 0.0568 | 830.0 |
| 非密閉空気層 | - | - | 0.090 | 0.0 |
| せっこうボード | 0.0125 | 0.22 | 0.0568 | 830.0 |

#### Function

In [14]:
def get_inner_wall_layers():
    return [
        {'Name': 'GPB', 'Cond': 0.22, 'Thick': 0.0125, 'SpecH': 830.0},
        {'Name': 'GPB', 'Cond': 1/0.09, 'Thick': 1, 'SpecH': 0},
        {'Name': 'GPB', 'Cond': 0.22, 'Thick': 0.0125, 'SpecH': 830.0}
    ]

### 3) 内壁等面積の推定

#### Outline

$$
A_{iw,MR-OR} = \left \{
\begin{array}{ll}
0.0 & ( A_{MR} = 0.0 \quad or \quad A_{OR} = 0.0 ) \\
\displaystyle
\max \left( 0.0, \frac{ A_{iw,MR} + A_{iw,OR} - A_{iw,NR} }{2} \right) & ( A_{MR} > 0.0 \quad and \quad A_{OR} > 0.0 )
\end{array}
\right.
$$

$$
A_{iw,MR-NR} = \left \{
\begin{array}{ll}
0.0 & ( A_{MR} = 0.0 \quad or \quad A_{NR} = 0.0 ) \\
\displaystyle
\max \left( 2.0, \frac{ A_{iw,MR} + A_{iw,NR} - A_{iw,OR} }{2} \right) & ( A_{MR} > 0.0 \quad and \quad A_{NR} > 0.0 )
\end{array}
\right.
$$

$$
A_{iw,NR-OR} = \left \{
\begin{array}{ll}
0.0 & ( A_{NR} = 0.0 \quad or \quad A_{OR} = 0.0 ) \\
\displaystyle
\max \left( 2.0, \frac{ A_{iw,NR} + A_{iw,OR} - A_{iw,MR} }{2} \right) & ( A_{NR} > 0.0 \quad and \quad A_{OR} > 0.0 )
\end{array}
\right.
$$

$ \because $

$$
A_{iw,MR} =A_{iw,MR-OR}+A_{iw,MR-NR}
$$

$$
A_{iw,OR} =A_{iw,MR-OR}+A_{iw,OR-NR}
$$

$$
A_{iw,NR} =A_{iw,MR-OR}+A_{iw,OR-NR}
$$

$A_{iw,MR-OR}$: 主たる居室とその他の居室の間の内壁等の面積, m<sup>2</sup>  
$A_{iw,MR-NR}$: 主たる居室と非居室の間の内壁等の面積, m<sup>2</sup>  
$A_{iw,OR-NR}$: その他の居室と非居室の間の内壁等の面積, m<sup>2</sup>  
$A_{iw,MR}$: 主たる居室の内壁等の面積, m<sup>2</sup>  
$A_{iw,OR}$: その他の居室の内壁等の面積, m<sup>2</sup>  
$A_{iw,NR}$: 非居室の内壁等の面積, m<sup>2</sup>  

$$
A_{iw,MR} = \left \{
\begin{array}{ll}
0 & ( A_{MR} = 0 ) \\
A_{w,MR} - A_{ow,MR} = \max \left ( 0, 2 \cdot A_{MR} + 4 \cdot ar_{MR} \cdot h \sqrt {A_{MR}} - A_{ow,MR} \right ) & ( A_{MR} > 0 )
\end{array}
\right.
$$

$$
A_{iw,OR} = \left \{
\begin{array}{ll}
0 & ( A_{OR} = 0 ) \\
A_{w,OR} - A_{ow,OR} = \max \left ( 0, 2 \cdot A_{OR} + 4 \cdot ar_{NR} \cdot h \sqrt {A_{OR}} - A_{ow,OR} \right ) & ( A_{OR} > 0 )
\end{array}
\right.
$$

$$
A_{iw,NR} = \left \{
\begin{array}{ll}
0 & ( A_{NR} = 0 ) \\
A_{w,NR} - A_{ow,NR} = \max \left ( 0, 2 \cdot A_{NR} + 4 \cdot ar_{OR} \cdot h \sqrt {A_{NR}} - A_{ow,NR} \right ) & ( A_{NR} > 0 )
\end{array}
\right.
$$

$A_{w,MR}$: 主たる居室の外皮と、主たる居室と床下空間の間の床と、主たる居室の内壁等の面積の合計, m<sup>2</sup>  
$A_{w,OR}$: その他の居室の外皮と、その他の居室と床下空間の間の床と、その他の居室の内壁等の面積の合計, m<sup>2</sup>  
$A_{w,NR}$: 非居室の外皮と、非居室と床下空間の間の床と、非居室の内壁等の面積の合計, m<sup>2</sup>  
$A_{ow,MR}$: 主たる居室の外皮と、主たる居室と床下空間の間の床の面積の合計, m<sup>2</sup>  
$A_{ow,OR}$: その他の居室の外皮と、その他の居室と床下空間の間の床の面積の合計, m<sup>2</sup>  
$A_{ow,NR}$: 非居室の外皮と、非居室と床下空間の間の床の面積の合計, m<sup>2</sup>  
$ar_{MR}$: 主たる居室の周長を求める際の係数, -  
$ar_{OR}$: その他の居室の周長を求める際の係数, -  
$ar_{NR}$: 非居室の周長を求める際の係数, -  
$h$: 階高, m

#### Function

##### 内壁等の面積を推定する関数

In [15]:
def get_area_inner_wall(d):

    # 居室の外皮と床下空間との間の床の面積の合計を取得
    def get_A_ow(d):

        # 室用途別の内壁等の面積
        d_ow=[]
        for y in ['Walls', 'Windows', 'Doors', 'InnerWalls']:
            if (y in d) == True :
                d_ow.extend(d[y])
        
        A_ow_MR = sum(y['area'] for y in d_ow if y['space'] == 'main')
        A_ow_OR = sum(y['area'] for y in d_ow if y['space'] == 'other')
        A_ow_NR = sum(y['area'] for y in d_ow if y['space'] == 'nonliving')
        
        return A_ow_MR, A_ow_OR, A_ow_NR
    
    # 内壁等の面積の取得
    def get_A_iw(A_f, ar, A_ow):
        
        h = 2.8 # 階高, m
        
        return max(0.0, 2.0 * A_f + 4 * h * ar * A_f**0.5 - A_ow) if A_f > 0.0 else 0.0
        
    # 室用途間の内壁等の面積の推定
    # 室用途床面積が0㎡の場合に、室間の内壁等の面積を0㎡とする
    def get_A_iw_cor(A_s1, A_s2, A_iw_min, A_iw_s1, A_iw_s2, A_iw_os):
        if A_s1 == 0.0 or A_s2 == 0.0:
            return 0.0
        else:
            return max(A_iw_min, (A_iw_s1 + A_iw_s2 - A_iw_os)/2)

    # 周長を求める際の係数, -
    ar_MR, ar_OR, ar_NR = 1.2, 1.4, 1.4

    # 用途別床面積の取得
    A_MR, A_OR, A_NR = get_area_floor(d)
 
    # 居室の外皮と床下空間との間の床の面積の合計を取得
    A_ow_MR, A_ow_OR, A_ow_NR = get_A_ow(d)

    # 内壁等の面積の取得
    A_iw_space_MR, A_iw_space_OR, A_iw_space_NR = get_A_iw(A_MR, ar_MR, A_ow_MR), get_A_iw(A_OR, ar_OR, A_ow_OR), get_A_iw(A_NR, ar_NR, A_ow_NR)

    # 室間の内壁等の面積の最小値, m2
    A_iw_min_MR_to_OR, A_iw_min_MR_to_NR, A_iw_min_OR_to_NR = 0.0, 2.0, 2.0
    
    
    # 室用途間の内壁等の面積の推定
    return {
        'main to other'      : get_A_iw_cor(A_MR, A_OR, A_iw_min_MR_to_OR, A_iw_space_MR, A_iw_space_OR, A_iw_space_NR),
        'main to nonliving'  : get_A_iw_cor(A_MR, A_NR, A_iw_min_MR_to_NR, A_iw_space_MR, A_iw_space_NR, A_iw_space_OR),
        'other to nonliving' : get_A_iw_cor(A_OR, A_NR, A_iw_min_OR_to_NR, A_iw_space_OR, A_iw_space_NR, A_iw_space_MR)
    }

#### Example

In [16]:
d = {
    'Common': {
        'Region': 6,
        'MainOccupantRoomFloorArea': 60.0,
        'OtherOccupantRoomFloorArea': 60.0,
        'TotalFloorArea': 120.0
    },
    'Walls': [
        { 'area': 60, 'direction': 'Top', 'name': 'Ceiling', 'space': 'main', 'structure': 'wood', 'type': 'ceiling' },
        { 'area': 15, 'direction': 'Bottom', 'name': 'Floor', 'space': 'other', 'structure': 'wood', 'type': 'floor' },
    ],
    'Windows': [
        { 'area': 30, 'direction': 'SW', 'name': 'WindowSW', 'space': 'main', 'type': 'single' },
        { 'area': 75, 'direction': 'Bottom', 'name': 'WindowBottom', 'space': 'other', 'type': 'single' },
    ],
    'Doors': [
        { 'area': 15, 'direction': 'NW', 'name': 'DoorNW', 'space': 'nonliving' },
        { 'area': 15, 'direction': 'NE', 'name': 'DoorNE', 'space': 'other' }
    ],
    'Earthfloors': [
        { 'area': 40, 'name': 'other', 'space': 'underfloor' },
        { 'area': 20, 'name': 'entrance', 'space': 'nonliving' }
    ]
}

get_area_inner_wall(d)

{'main to other': 135.28127504155998,
 'main to nonliving': 0.0,
 'other to nonliving': 0.0}

### 3) 内壁の構築

#### Function

In [17]:
def get_innerwall_wall(d):

    # 用途と複数用途間の対応関係の定義
    correspondence = {
        'main to other': ['main', 'other' ],
        'main to nonliving': ['main', 'nonliving' ],
        'other to nonliving': ['other', 'nonliving' ] 
    }  

    # 内壁面積の推定
    areas_innerwall = get_area_inner_wall(d)
    
    # 壁体名称、部位面積を更新し、「壁体が属する空間種類」「隣接空間種類」を追加する
    iws = []

    iws.extend([ {
        'Name' : 'Innerwall_' + c,
        'Type' : 'InnerWall',
        'Direction' : 'Vertical',
        'Layers' : get_inner_wall_layers(),
        'Area' : areas_innerwall[c],
        'Space' : correspondence[c][0],
        'NextSpace' : correspondence[c][1]
    } for c in correspondence.keys()
    ])

    iws.extend([ {
        'Name' : 'Innerwall_' + c,
        'Type' : 'InnerWall',
        'Direction' : 'Vertical',
        'Layers' : get_inner_wall_layers(),
        'Area' : areas_innerwall[c],
        'Space' : correspondence[c][1],
        'NextSpace' : correspondence[c][0]
    } for c in correspondence.keys()
    ])

    return iws

## 8.3. InnerWalls Element - 統合

#### Function

In [18]:
def convert_innerwall( d ):

    innerwalls =[]
    
    # 室と床下空間の間の床の推定
    innerwalls.extend(get_innerwall_floor(d))
    
    # 内壁等の推定              
    innerwalls.extend(get_innerwall_wall(d))
            
    return innerwalls

#### Example

In [19]:
d = {
    'Common': {'MainOccupantRoomFloorArea': 30.0, 'OtherOccupantRoomFloorArea': 60.0, 'Region': 6, 'TotalFloorArea': 120.0},
    'Doors': [
        {'area': 0.63, 'direction': 'NW', 'name': 'DoorNW_main', 'space': 'main'},
        {'area': 1.26, 'direction': 'NW', 'name': 'DoorNW_other', 'space': 'other'},
        {'area': 0.63, 'direction': 'NW', 'name': 'DoorNW_nonliving', 'space': 'nonliving'},
        {'area': 0.54, 'direction': 'NE', 'name': 'DoorNE_main', 'space': 'main'},
        {'area': 1.08, 'direction': 'NE', 'name': 'DoorNE_other', 'space': 'other'},
        {'area': 0.54, 'direction': 'NE', 'name': 'DoorNE_nonliving', 'space': 'nonliving'}
     ],
    'EarthfloorPerimeters': [
        {'direction': 'NW', 'length': 2.43, 'name': 'other_NW', 'psi': 1.8, 'space': 'underfloor'},
        {'direction': 'NE', 'length': 2.43, 'name': 'other_NE', 'psi': 1.8, 'space': 'underfloor'}
    ],
    'Earthfloors': [
        {'area': 5.0, 'name': 'other', 'space': 'underfloor'},
        {'area': 5.0, 'name': 'entrance', 'space': 'underfloor'}
    ],
    'Walls': [
        {'area': 16.95, 'direction': 'top', 'name': 'Ceiling_main', 'space': 'main', 'structure': 'wood', 'type': 'ceiling'},
        {'area': 33.9, 'direction': 'top', 'name': 'Ceiling_other', 'space': 'other', 'structure': 'wood', 'type': 'ceiling'},
        {'area': 16.95, 'direction': 'top', 'name': 'Ceiling_nonliving', 'space': 'nonliving', 'structure': 'wood', 'type': 'ceiling'},
        {'area': 10.1575, 'direction': 'SW', 'name': 'Wall_SW_main', 'space': 'main', 'structure': 'wood', 'type': 'wall'},
        {'area': 20.315, 'direction': 'SW', 'name': 'Wall_SW_other', 'space': 'other', 'structure': 'wood', 'type': 'wall'},
        {'area': 10.1575, 'direction': 'SW', 'name': 'Wall_SW_nonliving', 'space': 'nonliving', 'structure': 'wood', 'type': 'wall'}
    ],
    'Windows': [
        {'area': 7.5625, 'direction': 'SW', 'name': 'WindowSW_main', 'space': 'main', 'type': 'single'},
        {'area': 15.125, 'direction': 'SW', 'name': 'WindowSW_other', 'space': 'other', 'type': 'single'},
        {'area': 7.5625, 'direction': 'SW', 'name': 'WindowSW_nonliving',  'space': 'nonliving', 'type': 'single'},
         {'area': 0.7925, 'direction': 'NW', 'name': 'WindowNW_main', 'space': 'main', 'type': 'single'},
        {'area': 1.585, 'direction': 'NW', 'name': 'WindowNW_other', 'space': 'other', 'type': 'single'},
        {'area': 0.7925, 'direction': 'NW', 'name': 'WindowNW_nonliving', 'space': 'nonliving', 'type': 'single'}
        ]
}

convert_innerwall( d )

[{'Name': 'Innerfloor_main',
  'Type': 'InnerFloor',
  'Direction': 'Horizontal',
  'Area': 2.5,
  'Layers': [{'Name': 'PED', 'Cond': 0.16, 'Thick': 0.012, 'SpecH': 720.0}],
  'Space': 'main',
  'NextSpace': 'Underfloor'},
 {'Name': 'Innerfloor_other',
  'Type': 'InnerFloor',
  'Direction': 'Horizontal',
  'Area': 5.0,
  'Layers': [{'Name': 'PED', 'Cond': 0.16, 'Thick': 0.012, 'SpecH': 720.0}],
  'Space': 'other',
  'NextSpace': 'Underfloor'},
 {'Name': 'Innerfloor_nonliving',
  'Type': 'InnerFloor',
  'Direction': 'Horizontal',
  'Area': 2.5,
  'Layers': [{'Name': 'PED', 'Cond': 0.16, 'Thick': 0.012, 'SpecH': 720.0}],
  'Space': 'nonliving',
  'NextSpace': 'Underfloor'},
 {'Name': 'Innerfloor_main',
  'Type': 'InnerFloor',
  'Direction': 'Horizontal',
  'Area': 2.5,
  'Layers': [{'Name': 'PED', 'Cond': 0.16, 'Thick': 0.012, 'SpecH': 720.0}],
  'Space': 'Underfloor',
  'NextSpace': 'main'},
 {'Name': 'Innerfloor_other',
  'Type': 'InnerFloor',
  'Direction': 'Horizontal',
  'Area': 5.0

## 3. Lv4入力情報のコンバート（統合）
- 1)共通情報、2)開口部以外の不透明部位、3)開口部の透明部位、4)開口部の不透明部位、5) 線熱橋、6)土間床等の外周部の線熱橋、7)土間床、8)内壁等の各コンバート関数を統合し、Lv3の入力情報をLv4の入力情報にコンバートする。

### Function

In [20]:
def convert(d):
    d_lv4_input = {}
    d_lv4_input['Common'] = convert_common(d)
    if ('Walls' in d) == True:
        d_lv4_input['Walls'] = convert_wall(d)
    if ('Windows' in d) == True:
        d_lv4_input['Windows'] = convert_window(d)
    if ('Doors' in d) == True:
        d_lv4_input['Doors'] = convert_door(d)
    if ('EarthfloorPerimeters' in d) == True:
        d_lv4_input['EarthfloorPerimeters'] = convert_earthfloorperimeter(d)
    if ('Earthfloors' in d) == True:
        d_lv4_input['Earthfloors'] = convert_earthfloor(d)
    if ('Heatbridges' in d) == True:
        d_lv4_input['Heatbridges'] = convert_heatbridge(d)
    d_lv4_input['InnerWalls'] = convert_innerwall(d)

    return d_lv4_input

### Example

In [21]:
d = {
    'Common': {'MainOccupantRoomFloorArea': 30.0, 'OtherOccupantRoomFloorArea': 60.0, 'Region': 6, 'TotalFloorArea': 120.0},
    'Doors': [
        {'area': 0.63, 'direction': 'NW', 'name': 'DoorNW_main', 'space': 'main'},
        {'area': 1.26, 'direction': 'NW', 'name': 'DoorNW_other', 'space': 'other'},
        {'area': 0.63, 'direction': 'NW', 'name': 'DoorNW_nonliving', 'space': 'nonliving'},
        {'area': 0.54, 'direction': 'NE', 'name': 'DoorNE_main', 'space': 'main'},
        {'area': 1.08, 'direction': 'NE', 'name': 'DoorNE_other', 'space': 'other'},
        {'area': 0.54, 'direction': 'NE', 'name': 'DoorNE_nonliving', 'space': 'nonliving'}
     ],
    'EarthfloorPerimeters': [
        {'direction': 'NW', 'length': 2.43, 'name': 'other_NW', 'psi': 1.8, 'space': 'underfloor'},
        {'direction': 'NE', 'length': 2.43, 'name': 'other_NE', 'psi': 1.8, 'space': 'underfloor'}
    ],
    'Earthfloors': [
        {'area': 5.0, 'name': 'other', 'space': 'underfloor'},
        {'area': 5.0, 'name': 'entrance', 'space': 'underfloor'}
    ],
    'Walls': [
        {'area': 16.95, 'direction': 'top', 'name': 'Ceiling_main', 'space': 'main', 'structure': 'wood', 'type': 'ceiling'},
        {'area': 33.9, 'direction': 'top', 'name': 'Ceiling_other', 'space': 'other', 'structure': 'wood', 'type': 'ceiling'},
        {'area': 16.95, 'direction': 'top', 'name': 'Ceiling_nonliving', 'space': 'nonliving', 'structure': 'wood', 'type': 'ceiling'},
        {'area': 10.1575, 'direction': 'SW', 'name': 'Wall_SW_main', 'space': 'main', 'structure': 'wood', 'type': 'wall'},
        {'area': 20.315, 'direction': 'SW', 'name': 'Wall_SW_other', 'space': 'other', 'structure': 'wood', 'type': 'wall'},
        {'area': 10.1575, 'direction': 'SW', 'name': 'Wall_SW_nonliving', 'space': 'nonliving', 'structure': 'wood', 'type': 'wall'}
    ],
    'Windows': [
        {'area': 7.5625, 'direction': 'SW', 'name': 'WindowSW_main', 'space': 'main', 'type': 'single'},
        {'area': 15.125, 'direction': 'SW', 'name': 'WindowSW_other', 'space': 'other', 'type': 'single'},
        {'area': 7.5625, 'direction': 'SW', 'name': 'WindowSW_nonliving',  'space': 'nonliving', 'type': 'single'},
         {'area': 0.7925, 'direction': 'NW', 'name': 'WindowNW_main', 'space': 'main', 'type': 'single'},
        {'area': 1.585, 'direction': 'NW', 'name': 'WindowNW_other', 'space': 'other', 'type': 'single'},
        {'area': 0.7925, 'direction': 'NW', 'name': 'WindowNW_nonliving', 'space': 'nonliving', 'type': 'single'}
        ]
}

convert(d)

{'Common': {'MainOccupantRoomFloorArea': 30.0,
  'OtherOccupantRoomFloorArea': 60.0,
  'Region': 6,
  'TotalFloorArea': 120.0},
 'Walls': [{'area': 16.95,
   'direction': 'top',
   'name': 'Ceiling_main',
   'space': 'main',
   'structure': 'wood',
   'type': 'ceiling'},
  {'area': 33.9,
   'direction': 'top',
   'name': 'Ceiling_other',
   'space': 'other',
   'structure': 'wood',
   'type': 'ceiling'},
  {'area': 16.95,
   'direction': 'top',
   'name': 'Ceiling_nonliving',
   'space': 'nonliving',
   'structure': 'wood',
   'type': 'ceiling'},
  {'area': 10.1575,
   'direction': 'SW',
   'name': 'Wall_SW_main',
   'space': 'main',
   'structure': 'wood',
   'type': 'wall'},
  {'area': 20.315,
   'direction': 'SW',
   'name': 'Wall_SW_other',
   'space': 'other',
   'structure': 'wood',
   'type': 'wall'},
  {'area': 10.1575,
   'direction': 'SW',
   'name': 'Wall_SW_nonliving',
   'space': 'nonliving',
   'structure': 'wood',
   'type': 'wall'}],
 'Windows': [{'area': 7.5625,
   'di